In [1]:
# Django setup.
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'bookmark_db.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'

import django
django.setup()

# Django database connection and helper query functions.
from django.db import connection
cursor = connection.cursor()

def execute(query, params=None):
    cursor.execute(query, params)

def fetch_row(query):
    cursor.execute(query)
    return cursor.fetchone()

def fetch_rows(query):
    cursor.execute(query)
    columns = [col[0] for col in cursor.description]
    return [dict(zip(columns, row)) for row in cursor.fetchall()]

# Regular imports.
import calendar
import collections
from pprint import pprint
from urllib.parse import urlparse

from ipyvizzu import Data, Config, Style
from ipyvizzustory import Story, Slide, Step

# Common data.
min_year = fetch_row("SELECT EXTRACT(YEAR FROM MIN(created_on))::INT FROM bookmarks_bookmark;")[0]
max_year = fetch_row("SELECT EXTRACT(YEAR FROM MAX(created_on))::INT FROM bookmarks_bookmark;")[0]
MODE_AGGREGATE = 'aggregate'
MODE_BY_YEAR = 'by_year'
MODE = MODE_AGGREGATE

In [2]:
# bookmark_created_ons
min_year_start = f'{min_year}-01-01'
max_year_end = f'{max_year}-12-12'
execute("CREATE OR REPLACE VIEW all_dates AS SELECT EXTRACT(YEAR FROM series)::INT AS year, EXTRACT(MONTH FROM series)::INT AS month FROM GENERATE_SERIES(%s::TIMESTAMP, %s, '1 MONTH') AS series;", [min_year_start, max_year_end])
execute("CREATE OR REPLACE VIEW all_data AS SELECT EXTRACT(YEAR FROM created_on)::INT AS year, EXTRACT(MONTH FROM created_on)::INT AS month, COUNT(*) FROM bookmarks_bookmark GROUP BY year, month ORDER BY year, month;")
bookmark_created_ons = fetch_rows("SELECT year, month, COALESCE(count, 0) AS count FROM all_dates FULL OUTER JOIN all_data USING (year, month);")
for bookmark_created_on in bookmark_created_ons:
    bookmark_created_on['month'] = calendar.month_name[bookmark_created_on['month']][:3]
pprint('bookmark_created_ons:')
pprint(bookmark_created_ons[:3])
data = Data()
data.add_series('Year', [str(x['year']) for x in bookmark_created_ons], type='dimension')
data.add_series('Month', [x['month'] for x in bookmark_created_ons], type='dimension')
data.add_series('Count', [x['count'] for x in bookmark_created_ons], type='measure')
story = Story(data=data)
story.set_feature('tooltip', True)
slide = Slide(
    Step(
        Config({
            'channels': {
                'x': None,
                'y': 'Count',
                'label': 'Count',
            },
            'title': 'Bookmarks',
        }),
    ),
)
story.add_slide(slide)
slide = Slide(
    Step(
        Config({
            'channels': {
                'x': 'Year',
                'y': 'Count',
                'label': 'Count',
                'lightness': 'Count',
            },
            'title': 'Bookmarks by year',
        }),
    ),
)
story.add_slide(slide)
slide = Slide(
    Step(
        Config({
            'channels': {
                'x': 'Month',
                'y': 'Count',
                'label': 'Count',
                'lightness': 'Count',
            },
            'title': 'Bookmarks by month',
        }),
    ),
)
story.add_slide(slide)
slide = Slide(
    Step(
        Config({
            'channels': {
                'x': 'Month',
                'y': 'Count',
                'label': 'Count',
                'lightness': None,
            },
            'geometry': 'area',
            'title': 'Bookmarks by month',
        }),
    ),
)
story.add_slide(slide)
slide = Slide(
    Step(
        Config({
            'channels': {
                'x': 'Month',
                'y': ['Count', 'Year'],
                'color': 'Year',
                'label': None,
            },
            'geometry': 'area',
            'title': 'Bookmarks by month',
        }),
    ),
)
story.add_slide(slide)
slide = Slide()
for year in range(min_year, max_year + 1):
    step = Step(
        Config({
            'channels': {
                'x': 'Month',
                'y': ['Count', 'Year'],
                'color': 'Year',
                'label': None,
            },
            'geometry': 'area',
            'title': f'Bookmarks by month {min_year} - {year}',
        }),
        Data.filter(f"record['Year'] <= '{year}'"),
    )
    slide.add_step(step)
story.add_slide(slide)
slide = Slide(
    Step(
        Config({
            'channels': {
                'x': 'Month',
                'y': 'Count',
                'color': 'Year',
                'label': None,
            },
            'geometry': 'line',
            'title': 'Bookmarks by month',
        }),
        Data.filter(None),
    ),
)
story.add_slide(slide)
slide = Slide(
    Step(
        Config({
            'channels': {
                'x': 'Month',
                'y': 'Count',
                'color': 'Year',
                'label': None,
            },
            'geometry': 'line',
            'title': 'Bookmarks by month 2013 - 2014',
        }),
        Data.filter(f"record['Year'] >= '2013' && record['Year'] <= '2014'"),
    ),
)
story.add_slide(slide)
slide = Slide(
    Step(
        Config({
            'channels': {
                'x': 'Month',
                'y': ['Count', 'Year'],
                'color': 'Year',
                'label': None,
            },
            'geometry': 'rectangle',
            'title': 'Bookmarks by month 2013 - 2014',
        }),
        Data.filter(f"record['Year'] >= '2013' && record['Year'] <= '2014'"),
    ),
)
story.add_slide(slide)
slide = Slide(
    Step(
        Config({
            'channels': {
                'x': 'Count',
                'y': 'Year',
                'color': 'Year',
                'label': None,
            },
            'geometry': 'rectangle',
            'reverse': True,
            'title': 'Bookmarks by month 2013 vs 2014',
        }),
        Data.filter(f"record['Year'] >= '2013' && record['Year'] <= '2014'"),
    ),
)
story.add_slide(slide)
slide = Slide(
    Step(
        Config({
            'channels': {
                'x': ['Count', 'Month'],
                'y': 'Year',
                'color': 'Month',
                'label': 'Month',
            },
            'geometry': 'rectangle',
            'reverse': True,
            'title': 'Bookmarks by month 2013 vs 2014',
        }),
        Data.filter(f"record['Year'] >= '2013' && record['Year'] <= '2014'"),
    ),
)
story.add_slide(slide)
story.play()

'bookmark_created_ons:'
[{'count': 0, 'month': 'Jan', 'year': 2023},
 {'count': 0, 'month': 'Feb', 'year': 2023},
 {'count': 0, 'month': 'Mar', 'year': 2023}]


In [3]:
# bookmark_domains
bookmark_domains = fetch_rows("SELECT EXTRACT(YEAR FROM created_on)::INT AS year, url FROM bookmarks_bookmark ORDER BY year;")
for bookmark_domain in bookmark_domains:
    domain = urlparse(bookmark_domain['url']).netloc
    if domain.startswith('www.'):
        domain = domain[4:]
    bookmark_domain['domain'] = domain
    del bookmark_domain['url']
bookmark_domains = [x for x in bookmark_domains if x['domain'] != '']
pprint('bookmark_domains:')
pprint(bookmark_domains[:3])
top_bookmark_domains = []
for year in range(min_year, max_year + 1):
    domain_counter = collections.Counter([x['domain'] for x in bookmark_domains if (x['year'] <= year if MODE == MODE_AGGREGATE else x['year'] == year)])
    for domain, count in domain_counter.most_common(10):
        top_bookmark_domains.append({
            'year': year,
            'domain': domain,
            'count': count,
        })
pprint('top_bookmark_domains:')
pprint(top_bookmark_domains[:3])
data = Data()
data.add_series('Year', [str(x['year']) for x in top_bookmark_domains], type='dimension')
data.add_series('Domain', [x['domain'] for x in top_bookmark_domains], type='dimension')
data.add_series('Count', [x['count'] for x in top_bookmark_domains], type='measure')
story = Story(data=data)
for year in range(min_year, max_year + 1):
    slide = Slide(
        Step(
            Config({
                'channels': {
                    'x': 'Count',
                    'y': 'Domain',
                    'color': 'Domain',
                    'label': 'Count',
                },
                'sort': 'byValue',
                'title': f'Top 10 domains {min_year} - {year}' if MODE == MODE_AGGREGATE else f'Top 10 domains {year}',
            }),
            Data.filter(f"record['Year'] == '{year}'"),
            Style({
                'plot': {'paddingLeft': 250},
            }),
        ),
    )
    story.add_slide(slide)
story.play()

'bookmark_domains:'
[{'domain': 'old-bookmark-c.db', 'year': 2023},
 {'domain': 'old-bookmark-a.db', 'year': 2023},
 {'domain': 'old-bookmark-b.db', 'year': 2023}]
'top_bookmark_domains:'
[{'count': 1, 'domain': 'old-bookmark-c.db', 'year': 2023},
 {'count': 1, 'domain': 'old-bookmark-a.db', 'year': 2023},
 {'count': 1, 'domain': 'old-bookmark-b.db', 'year': 2023}]


In [4]:
# bookmark_labels
bookmark_labels = fetch_rows("SELECT EXTRACT(YEAR FROM b.created_on)::INT AS year, LOWER(l.name) AS label FROM bookmarks_bookmark b JOIN bookmarks_bookmark_labels bl ON b.id = bl.bookmark_id JOIN bookmarks_label l ON l.id = bl.label_id WHERE lower(l.name) != 'bookmarks bar' ORDER BY year, label;")
pprint('bookmark_labels:')
pprint(bookmark_labels[:3])
top_bookmark_labels = []
for year in range(min_year, max_year + 1):
    label_counter = collections.Counter([x['label'] for x in bookmark_labels if (x['year'] <= year if MODE == MODE_AGGREGATE else x['year'] == year)])
    for label, count in label_counter.most_common(10):
        top_bookmark_labels.append({
            'year': year,
            'label': label,
            'count': count,
        })
pprint('top_bookmark_labels:')
pprint(top_bookmark_labels[:3])
data = Data()
data.add_series('Year', [str(x['year']) for x in top_bookmark_labels], type='dimension')
data.add_series('Label', [x['label'] for x in top_bookmark_labels], type='dimension')
data.add_series('Count', [x['count'] for x in top_bookmark_labels], type='measure')
story = Story(data=data)
for year in range(min_year, max_year + 1):
    slide = Slide(
        Step(
            Config({
                'channels': {
                    'x': 'Count',
                    'y': 'Label',
                    'color': 'Label',
                    'label': 'Count',
                },
                'sort': 'byValue',
                'title': f'Top 10 labels {min_year} - {year}' if MODE == MODE_AGGREGATE else f'Top 10 labels {year}',
            }),
            Data.filter(f"record['Year'] == '{year}'"),
            Style({
                'plot': {'paddingLeft': 250},
            }),
        ),
    )
    story.add_slide(slide)
story.play()

'bookmark_labels:'
[{'label': 'new folder d', 'year': 2023},
 {'label': 'new folder e', 'year': 2023},
 {'label': 'old folder a', 'year': 2023}]
'top_bookmark_labels:'
[{'count': 2, 'label': 'old folder a', 'year': 2023},
 {'count': 1, 'label': 'new folder d', 'year': 2023},
 {'count': 1, 'label': 'new folder e', 'year': 2023}]
